# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

# 1
1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

``` sql
{{ config(materialized='table') }}

SELECT
    EXTRACT(YEAR FROM order_approved_at) AS ano,
    COUNT(*) AS contagem_ordens
FROM 
    `dbt_shop.orders`
GROUP BY 
    ano
ORDER BY
    ano
```

dbt run --models ativ.A3_01_orders_by_year

![Orders by year](dbt_atividade_3_imagens/01_orders_by_year.png)

# 2
2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

``` sql
SELECT
    customer_state AS estado,
    COUNT(*) AS contagem_ordens
FROM 
    `dbt_shop.orders`
INNER JOIN
    `dbt_shop.customers`
ON
    `dbt_shop.orders`.customer_id = `dbt_shop.customers`.customer_id
GROUP BY 
    estado
ORDER BY
    estado
```

dbt run --models A3_02_orders_by_state

![Orders by state](dbt_atividade_3_imagens/02_orders_by_state.png)

# 3
3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

``` sql
SELECT
    estado
FROM
    `atividade_3.A3_02_orders_by_state`
WHERE
    contagem_ordens > CAST('{{ var("number_of_orders") }}' AS INTEGER)
ORDER BY
    estado

```

dbt run --models A3_03_state_by_order_count --vars "{'number_of_orders': 5000}"

![State by Order Count](dbt_atividade_3_imagens/03_state_by_order_count.png)

# 4
4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

Estados por região:
- Norte:
    AC
    AP
    AM
    PA
    RO
    RR
    TO

- Nordeste:
    AL
    BA
    CE
    MA
    PB
    PE
    PI
    RN
    SE

- Sudeste:
    ES
    MG
    RJ
    SP

- Centro-Oeste:
    DF
    GO
    MT
    MS

- Sul:
    PR
    RS
    SC

#### (Tentei criar uma tabela de domínio para auxiliar essa query por join e remover os cases/ifs, mas o GCP exigiu cadastrar método de pagamento para inserir dados brutos em uma nova tabela). Segue a query longa para contornar o problema

``` sql
WITH customer_count_by_state AS (
    SELECT
        customer_state,
        COUNT(*) AS customer_count
    FROM
        `dbt_shop.customers`
    GROUP BY
        customer_state
)
SELECT
    IF (
        customer_state = 'AC' OR customer_state = 'AP' OR customer_state = 'AM' OR customer_state = 'PA' OR customer_state = 'RO' OR customer_state = 'RR' OR customer_state = 'TO',
        'Norte',
        IF (
            customer_state = 'AL' OR customer_state = 'BA' OR customer_state = 'CE' OR customer_state = 'MA' OR customer_state = 'PB' OR customer_state = 'PE' OR customer_state = 'PI' OR customer_state = 'RN' OR customer_state = 'SE',
            'Nordeste',
            IF (
                customer_state = 'ES' OR customer_state = 'MG' OR customer_state = 'RJ' OR customer_state = 'SP',
                'Sudeste',
                IF (
                    customer_state = 'DF' OR customer_state = 'GO' OR customer_state = 'MT' OR customer_state = 'MS',
                    'Centro-Oeste',
                    IF (
                        customer_state = 'PR' OR customer_state = 'RS' OR customer_state = 'SC',
                        'Sul',
                        NULL
                    )
                )
            )
        )
    ) AS region_name,
    SUM(customer_count) AS customer_count
FROM
    customer_count_by_state
GROUP BY
    region_name
ORDER BY
    region_name


```

dbt run --models A3_04_customer_by_region

![Customer by Region](dbt_atividade_3_imagens/04_customer_by_region.png)

# 5
5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

``` sql
{{ config(materialized='table') }}

SELECT
    order_id,
    product_id,
    price + freight_value AS item_cost
FROM
    `dbt_shop.items`
GROUP BY
    order_id, product_id, item_cost
```

dbt run --models A3_05_item_cost

![Item Cost](dbt_atividade_3_imagens/05_item_cost.png)

# 6
6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

``` sql
SELECT
    o.order_id AS order_id,
    SUM(i.price + i.freight_value) AS total_cost
FROM
    `dbt_shop.items` i
INNER JOIN
    `dbt_shop.orders` o
ON o.order_id = i.order_id
WHERE
    o.order_status = 'shipped'
GROUP BY
    o.order_id
```

dbt run --models A3_06_orders_shipped_cost

![Orders Shipped Cost](dbt_atividade_3_imagens/06_orders_shipped_cost.png)